In [ ]:
import os
import pandas as pd
import numpy as np
import glob

path = f'/media/EXTHDD/UnfollowData/WeeklySnapshots/round1-15/round15/Tweets'
os.chdir(path)

user_data = []
tweet_data = []

for file in glob.glob("*.pkl"):
    f1 = pd.read_pickle(file)
    for j in f1:
        user_data.append([
            j.get('user').get('id_str', 0),
            j.get('user').get('verified', 0),
            j.get('user').get('created_at', 0),
            j.get('user').get('location', 0),
            j.get('user').get('description', 0),
            j.get('user').get('profile_image_url_https', 0),
            j.get('user').get('listed_count', 0),
            j.get('user').get('url', 0),
            j.get('user').get('favourites_count', 0),
            j.get('user').get('statuses_count', 0)
        ])

user_columns = ['id_str', 'verified', 'created_at', 'location', 'description',
                'profile_image_url_https', 'listed_count', 'url', 'favourites_count', 'statuses_count']

user_df = pd.DataFrame(user_data, columns=user_columns)

print(user_df.head())


In [ ]:
user_df=user_df.drop_duplicates()

In [ ]:
mega_user_set1 = pd.read_pickle("/media/EXTHDD/UnfollowData/WeeklySnapshots/round1-15/megaUserSet.pkl")
mega_user_str=set()
for i in mega_user_set1:
    mega_user_str.add(str(i))

user_id_str_set = set(user_df.id_str)
l = [i for i in mega_user_str if i not in user_id_str_set]


In [ ]:
import pandas as pd

# Create a new DataFrame with the same columns as user_df
new_df = pd.DataFrame(columns=user_df.columns)

# Set the 'id_str' column to the values in the list l
new_df['id_str'] = l

# Fill the other columns with 0
new_df.fillna(0, inplace=True)

# Concatenate the new DataFrame with user_df
user_df_extended = pd.concat([user_df, new_df], ignore_index=True)

In [ ]:
import pandas as pd

# Convert the 'created_at' column to datetime format
user_df_extended['created_at'] = pd.to_datetime(user_df_extended['created_at'], errors='coerce', format='%a %b %d %H:%M:%S %z %Y')

# Extract the year from the 'created_at' column
user_df_extended['year'] = user_df_extended['created_at'].dt.year


In [ ]:
user_df_extended=user_df_extended.drop(columns={'created_at'})
user_df_extended.year.fillna(2012, inplace=True)

In [ ]:
user_df_extended['profile_image_url_https'] = user_df_extended['profile_image_url_https'].apply(lambda x: 1 if isinstance(x, str) and x.strip() else 0)
user_df_extended['url'] = user_df_extended['url'].apply(lambda x: 1 if isinstance(x, str) and x.strip() else 0)
user_df_extended['location'] = user_df_extended['location'].apply(lambda x: 1 if isinstance(x, str) and x.strip() else 0)


In [ ]:
# Compute the mean of non-zero 'listed_count' values
mean_listed_count = user_df_extended[user_df_extended['listed_count'] != 0]['listed_count'].mean()

# Replace 0 values in the 'listed_count' column with the computed mean
user_df_extended['listed_count'] = user_df_extended['listed_count'].replace(0, mean_listed_count)
# Compute the mean of non-zero 'listed_count' values
mean_listed_count = user_df_extended[user_df_extended['favourites_count'] != 0]['favourites_count'].mean()

# Replace 0 values in the 'listed_count' column with the computed mean
user_df_extended['favourites_count'] = user_df_extended['favourites_count'].replace(0, mean_listed_count)

# Compute the mean of non-zero 'listed_count' values
mean_listed_count = user_df_extended[user_df_extended['statuses_count'] != 0]['statuses_count'].mean()

# Replace 0 values in the 'listed_count' column with the computed mean
user_df_extended['statuses_count'] = user_df_extended['statuses_count'].replace(0, mean_listed_count)



In [ ]:
user_df_extended.to_csv("/home/leanna/unfollow_prediction/Markov_chain/dd5.csv")

In [ ]:
user_df_extended1=pd.DataFrame(pd.read_csv("/home/leanna/unfollow_prediction/Markov_chain/LIWC-22 Results - dd - LIWC Analysis (1).csv", low_memory = False))

In [ ]:
user_df_extended1= user_df_extended1.drop(columns={'Unnamed: 0','ColumnID'})

In [ ]:
import pandas as pd

# Custom function to find the top 2 highest scores
def find_top_two(row):
    return row.nlargest(2).index

# Convert the relevant columns to a numeric data type
columns = ['Culture', 'politic', 'ethnicity', 'tech', 'Lifestyle', 'leisure', 'home', 'work',
           'money', 'relig', 'Physical', 'health', 'illness', 'wellness', 'mental',
           'substances', 'sexual', 'food', 'death']

user_df_extended1[columns] = user_df_extended1[columns].apply(pd.to_numeric, errors='coerce')
user_df_extended1[columns] = user_df_extended1[columns].fillna(-1)

# Find the top 2 highest scores for each row
user_df_extended1[['related_1', 'related_2']] = user_df_extended1[columns].apply(find_top_two, axis=1, result_type='expand')

# Drop the original columns
user_df_extended1.drop(columns, axis=1, inplace=True)


In [ ]:
user_df_extended1.drop('Text', axis=1, inplace=True)
user_df_extended1.replace('E', 1, inplace=True)
user_df_extended1['url'].fillna(0, inplace= True)

In [ ]:
user_df_extended1.dtypes

In [ ]:
import pandas as pd

# Example DataFrame
data = {'A': [1, 2, 'E', 4, 5],
        'B': [10, 'Not a number', 30, 40, 50]}

df = pd.DataFrame(data)

# Custom function to check if a value is numeric
def to_numeric_or_zero(value):
    if pd.to_numeric(value, errors='coerce') is not None:
        return pd.to_numeric(value, errors='coerce')
    else:
        return 0

# Apply the custom function to each element in the DataFrame
user_df_extended1 = user_df_extended1.applymap(to_numeric_or_zero)

In [ ]:
user_df_extended1['profile_image_url_https']=user_df_extended1['profile_image_url_https'].astype(float)
user_df_extended1['listed_count']=user_df_extended1['listed_count'].astype(float)

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

# Fit the encoder to the DataFrame's 'A' column and transform the data
user_df_extended1['related_1'] = encoder.fit_transform(user_df_extended1['related_1'])
user_df_extended1['related_2']= encoder.fit_transform(user_df_extended1['related_2'])



In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
id_str=user_df_extended1.id_str
# Fit the scaler to the DataFrame and transform the data
normalized_data = scaler.fit_transform(user_df_extended1)

# Convert the normalized data back to a DataFrame
normalized_df = pd.DataFrame(normalized_data, columns=user_df_extended1.columns)

In [ ]:
normalized_df.drop(columns={"id_str"}, inplace= True)

In [ ]:
cols = ['id_str'] + [col for col in normalized_df.columns if col != 'id_str']
normalized_df = normalized_df[cols]

In [ ]:
normalized_df.to_csv("/home/leanna/unfollow_prediction/Markov_chain/dataset/nodes_15csv")